In [1]:

import os
import pickle
import sys
sys.path.append(os.path.abspath(os.path.join("..", "..", "code")))

import pandas as pd

from identify_watersheds import WatershedIdentifier
from utils import list_streamgage_files


DATA_DIR = os.path.join("..", "..", "data")
TIF_DIR = os.path.join(DATA_DIR, "gage_tiff_files")
files_df = list_streamgage_files(TIF_DIR)
files_df = files_df[files_df["latest"]]
files_df

,file,gage_id,timestamp,latest
3,11402000_30m_elv_2022_09_29__13_59_03.tif,11402000,2022-09-29 13:59:03,True
4,11318500_30m_elv_2022_09_29__13_57_45.tif,11318500,2022-09-29 13:57:45,True
0,11266500_30m_elv_2022_09_29__13_56_10.tif,11266500,2022-09-29 13:56:10,True
7,11208000_30m_elv_2022_09_29__13_54_21.tif,11208000,2022-09-29 13:54:21,True
1,11202710_30m_elv_2022_09_29__13_52_05.tif,11202710,2022-09-29 13:52:05,True
6,11189500_30m_elv_2022_09_29__18_01_00.tif,11189500,2022-09-29 18:01:00,True
2,11185500_30m_elv_2022_09_28__23_35_26.tif,11185500,2022-09-28 23:35:26,True


In [2]:
gages = pd.read_csv(os.path.join(DATA_DIR, "target_gages.csv"), encoding="utf-8")
gages = gages.set_index(["site_no"])
gages

,agency_cd,station_nm,site_tp_cd,lat_va,long_va,dec_lat_va,dec_long_va,coord_meth_cd,coord_acy_cd,coord_datum_cd,...,peak_count_nu,qw_begin_date,qw_end_date,qw_count_nu,gw_begin_date,gw_end_date,gw_count_nu,sv_begin_date,sv_end_date,sv_count_nu
site_no,,,,,,,,,,,,,,,,,,,,,
11185500,USGS,KERN R NO 3 CN NR KERNVILLE CA,ST,355420.0,1182800.0,35.905504,-118.467586,M,F,NAD27,...,0.0,--,--,0.0,--,--,0.0,2008-05-13,2022-04-28,33.0
11189500,USGS,SF KERN R NR ONYX CA,ST,354415.0,1181022.0,35.737452,-118.173689,M,F,NAD27,...,101.0,1979-11-14,1980-08-05,4.0,--,--,0.0,1930-03-25,2022-08-30,1103.0
11202710,USGS,MF TULE R BL INTAKE AB SPRINGVILLE CA,ST,360941.0,1184231.0,36.161336,-118.709536,M,F,NAD27,...,29.0,--,--,0.0,--,--,0.0,1990-06-12,2022-04-27,137.0
11208000,USGS,MARBLE F KAWEAH R (R ONLY) A POTWISHA CAMP CA,ST,363119.0,1184754.0,36.521890,-118.799265,M,F,NAD27,...,53.0,1980-06-12,1980-10-08,5.0,--,--,0.0,1984-06-10,2021-11-16,62.0
11266500,USGS,MERCED R A POHONO BRIDGE NR YOSEMITE CA,ST,374301.0,1193955.0,37.716871,-119.666279,M,F,NAD27,...,105.0,1971-10-18,2004-06-10,17.0,--,--,0.0,1916-11-02,2022-08-10,827.0
11318500,USGS,SF MOKELUMNE R NR WEST POINT CA,ST,382206.0,1203240.0,38.368246,-120.545482,M,F,NAD27,...,88.0,--,--,0.0,--,--,0.0,1958-01-26,2022-05-10,191.0
11402000,USGS,SPANISH C AB BLACKHAWK C AT KEDDIE CA,ST,400011.0,1205712.0,40.002947,-120.954399,M,F,NAD27,...,88.0,1971-09-21,1972-09-06,13.0,--,--,0.0,1935-04-04,2022-09-07,563.0


In [3]:
watersheds = dict()
for ix, row in files_df.iterrows():
    f, gage = row["file"], row["gage_id"]
    lat, lon = gages.loc[int(gage)]["dec_lat_va"], gages.loc[int(gage)]["dec_long_va"]
    tif_fp = os.path.join(TIF_DIR, f)
    watersheds[gage] = WatershedIdentifier(tif_fp, lat, lon, 1)

/Users/tp/virtualenvs/discharge-estimation/lib/python3.9/site-packages/pysheds/io.py:142: UserWarning: No `nodata` value detected. Defaulting to 0.
  warnings.warn('No `nodata` value detected. Defaulting to 0.')
/Users/tp/virtualenvs/discharge-estimation/lib/python3.9/site-packages/pysheds/io.py:142: UserWarning: No `nodata` value detected. Defaulting to 0.
  warnings.warn('No `nodata` value detected. Defaulting to 0.')
/Users/tp/virtualenvs/discharge-estimation/lib/python3.9/site-packages/pysheds/io.py:142: UserWarning: No `nodata` value detected. Defaulting to 0.
  warnings.warn('No `nodata` value detected. Defaulting to 0.')
/Users/tp/virtualenvs/discharge-estimation/lib/python3.9/site-packages/pysheds/io.py:142: UserWarning: No `nodata` value detected. Defaulting to 0.
  warnings.warn('No `nodata` value detected. Defaulting to 0.')
/Users/tp/virtualenvs/discharge-estimation/lib/python3.9/site-packages/pysheds/io.py:142: UserWarning: No `nodata` value detected. Defaulting to 0.
  wa

In [4]:
{k: v.catchment_touches_grid_edge for k, v in watersheds.items()}

{'11402000': False,
 '11318500': False,
 '11266500': False,
 '11208000': False,
 '11202710': False,
 '11189500': False,
 '11185500': False}

In [5]:
{k: v.clipped_grid.bbox for k, v in watersheds.items()}

{'11402000': (-120.9545597455571,
  40.002698254069585,
  -120.95402075638664,
  40.00377623241053),
 '11318500': (-120.54546696516907,
  38.367135616273174,
  -120.54519747058383,
  38.36821359461412),
 '11266500': (-119.6663756281297,
  37.71630619292858,
  -119.66610613354446,
  37.71684518209905),
 '11208000': (-118.79914205284072,
  36.52109770740756,
  -118.79833356908502,
  36.52190619116327),
 '11202710': (-118.70966985054243,
  36.16078344694722,
  -118.70940035595719,
  36.16132243611769),
 '11189500': (-118.174453604264,
  35.73659896978599,
  -118.17337562592306,
  35.73740745354169),
 '11185500': (-118.4673942184154,
  35.90530258014363,
  -118.46712472383015,
  35.90611106389934)}